In [ ]:

from library import fun_api as api
from library import formula as fr
from library import mi_telegram as tl
from library import my_credentials as crd
from library import my_load_text as tx

import sys
import pandas as pd
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, CallbackContext  
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update ,Message,CallbackQuery
from telegram.replykeyboardremove import ReplyKeyboardRemove
import datetime
import telegram
import schedule
from datetime import datetime
from datetime import time
from datetime import date
from datetime import timedelta
import time

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

def bot_mafe(update,context):
    TOKEN,URL = crd.credentialstl()
    try:
        id_chat = update.message.chat.id
        texto = update.message.text
        nombre = update.message.chat.username
        if nombre is None:
            nombre = "Amig@"
        else:
            nombre = update.message.chat.username.replace("_"," ")
        mensaje_id = update.message.message_id
        nametxt = str(id_chat) + ".txt"
        level, sublevel, consec,aux5,aux6,aux7,aux8,aux9 = tx.collect_data(nametxt,id_chat)
        if texto == 'Toma realizada!' and level != 'vencido':
            nametxt2 = 'R-' + str(id_chat)  + '.txt'
            level, sublevel, consec,aux5,aux6,aux7,aux8,aux9 = tx.collect_data(nametxt2,id_chat)
            lista = level.split('-') 
            print(lista)
            for idp in lista:
                api.ActualizarTomaMedicamentos(str(idp),'SI','SI',str(consec))
            reply_markup = ReplyKeyboardRemove()
            update.message.reply_text(sublevel, reply_markup=reply_markup,parse_mode = "Markdown")    
            tx.registro(id_chat,"RECORDATORIOS","0","0","0","0","0","0") 
            level, sublevel, consec,aux5,aux6,aux7,aux8,aux9 = tx.collect_data(nametxt,id_chat) 
        elif texto == 'Toma realizada!' and level == 'vencido':
            update.message.reply_text(sublevel,parse_mode = "Markdown")           
        elif level == 'RECORDATORIOS' or level == '0' or ((update.message.text.upper().find("HOLA") > 0) or "0" == texto or "/start" in texto):
            exist_patients = api.Validate_existence_id_telegram(id_chat)
            if exist_patients == False:
                texto_respuesta = 'Hola ' + nombre + ', mi nombre es MAFE 👩‍💻 tu asistente para el programa de seguimiento farmacológico de *SIES Salud* a continuación se encuentran los términos y condiciones del programa,por favor léelos detenidamente.⚠️ Términos y condiciones ⚠️'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                tl.enviar_pdf(id_chat,'Habeas Data-Terminos y condiciones.pdf',TOKEN) 
                tl.button_send(update, context)
                tx.registro(id_chat,"TERMINOS","0","0","0","0","0","0")
                print('Exit 1')
            else:
                texto_respuesta = '¡Hola!🏅 *' + nombre + '* 🤗 tienes estos medicamentos pendientes por registrar Muchas gracias por tu colaboración'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                texto_respuesta = 'En el archivo adjunto se encuentra la información relacionada con tus medicamentos 💊. ¡Sigamos el camino hacia el empoderamiento de tu salud!'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")                
                fr.send_formula(id_chat,update) #aqui                
        elif level == 'REGISTRO':
            exist_patient,nombre  = api.Validate_existence_id_pac(texto)
            if exist_patient is False:
                if int(consec) >= 2:
                    texto_respuesta = "Lo siento 😥 ha alcanzado el número máximo de intentos. Digita cero para reiniciar"
                    tx.registro(id_chat,"0","0","0","0","0","0","0") 
                elif int(consec) ==1:
                    texto_respuesta = "Te queda un intento, verifica tu número de cedula"
                    tx.registro(id_chat,"REGISTRO","0","2","0","0","0","0")  
                elif int(consec) == 0:
                    texto_respuesta = 'Lo siento 😥 tu número de documento no coincide con nuestros registros, verifica por favor tu número de documento y vuélvelo a ingresar 📥' 
                    tx.registro(id_chat,"REGISTRO","0","1","0","0","0","0")  
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")                 
            else:
                api.TerminosPaciente(texto,'Si')
                api.saveidtelegram(texto,id_chat)
                texto_respuesta = '¡Genial!🏅 *' + nombre + '* 🤗 te doy la bienvenida, hoy comenzamos el camino para mejorar tu calidad de vida, ¡Ánimo!. A partir de hoy estaré para apoyarte en la toma de tus medicamentos 💊, recordándote la hora ⏰ de la toma ☑️. Muchas gracias por tu colaboración 😁 Ya haces parte del programa de seguimiento a la adherencia.'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                texto_respuesta = 'En el archivo adjunto se encuentra la información relacionada con tus medicamentos 💊 para que la consultes cuando desees 😄. ¡Sigamos el camino hacia el empoderamiento de tu salud!'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                fr.send_formula(id_chat,update) #aqui
        elif level == "transaccional":
            if texto.upper() == "NO" or (texto.upper().find("NO") > 0):
                texto_respuesta = '😱 ¿Por qué? Para nosotros es de vital importancia saber la razón, por favor escoge una de las siguientes opciones 📌'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                texto_respuesta = '''1. ⚠️ No tengo la fórmula actualizada ⚠️
    2. ⚠️ No hay existencias en farmacia ⚠️
    3. ⚠️No los he reclamado ⚠️'''
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                print('Exit 6')
            if texto == "1" or texto == "2" or texto == "3":
                ahora = datetime.now()
                fechaPorQue = ahora + timedelta(days=7)
                print(fechaPorQue)
                api.NoCuentaMedicamentos(id_chat,texto,str(fechaPorQue))                
                texto_respuesta= 'Muchas gracias por tu respuesta 🤗, es importante que tengas todos tus medicamentos lo más pronto posible, recuerda que tú bienestar depende de tí 👍'
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")
                tx.registro(id_chat,"0","0","0","0","0","0","0")
    #             api.TomaMedicamentos(str(id_chat),'0','NO',str(fechaPorQue),'NO','0','2')
                print('Exit 7')
            if texto.upper() == "SI" or (texto.upper().find("SI") > 0) or texto.upper() == "SÍ":
                texto_respuesta = '¡Genial! ✨ Pronto estarás recibiendo los recordatorios para la toma de tus medicamentos ⚠️, juntos lograremos tu bienestar 🤝 éxitos.'
                fr.elaborate_formula2(id_chat)
                ahora = datetime.now()
                now = str(ahora.strftime("%Y-%m-%d %H:%M:%S"))
                print(now)
                api.CuentaMedicamentos(id_chat,now) #guardar la información en caso de que el paciente cuente con todos los medicamentos
                update.message.reply_text(texto_respuesta,parse_mode = "Markdown")          
                tx.registro(id_chat,"0","0","0","0","0","0","0") 
                print('Exit 5')
        else:
            texto_respuesta = "Comando No reconocido \"" + texto + "\, Digite 0 para Reiniciar "
            update.message.reply_text(texto_respuesta,parse_mode = "Markdown")    
    except (ValueError):
        update.message.reply_text("Se generó un error, comuníquese con el admin")
    
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

def job():
    TOKEN,URL = crd.credentialstl()
    ahora = datetime.now()
    now = ahora.strftime("%Y-%m-%d %H:%M:%S")
    fecha = []
    fecha.append(now)
    df = pd.DataFrame()
    df['fecha'] = fecha
    df.to_csv('record_fecha.txt',index=None, header=None, sep=';')
    lpedidos = open('record_fecha.txt', "r").read()
    l = lpedidos.replace("\n",'')
    date1 = pd.to_datetime(l)  
    ffinal = date1 + timedelta(seconds= 300)
    programa_recordatorios = api.ConsultaTomaMedicamentos('123')
    if programa_recordatorios.empty:
        namepng = "Sin_datos.png"
    else:
        programa_recordatorios['medicamento'] = programa_recordatorios['NombreMedicamento'] + ' ' + programa_recordatorios['NombreFormaFarmaceutica'] 
        programa_recordatorios['fechaToma'] = pd.to_datetime(programa_recordatorios['fechaToma'])
        programa_recordatorios['hora'] = programa_recordatorios['fechaToma'].dt.hour
        programa_recordatorios['Minuto']= programa_recordatorios['fechaToma'].dt.minute
        filtered_df = programa_recordatorios[(programa_recordatorios['procesado'] == 'NO') & (programa_recordatorios['fechaToma'] <= str(ffinal)) & (programa_recordatorios['recordatorioTipo'] == '1')]
        lid_telegram = filtered_df['idTelegram'].unique()
        # print(lid_telegram,'lid_telegram')
        for telegram in lid_telegram:
            list_medicamentos = []
            df_telegram = filtered_df[(filtered_df['idTelegram'] == telegram)] 
            lfechas = df_telegram['fechaToma'].unique()
            print('lfechas',lfechas)
            for fecha in lfechas:
                dfx = df_telegram[(df_telegram['fechaToma'] == fecha)] 
                hora = dfx['hora'].unique()
                minuto = dfx['Minuto'].unique()  
                id_tel = dfx['idTelegram'].unique() 
                lnumeroEnvios = dfx['numeroEnvio'].unique()
                for numeroEnvio in lnumeroEnvios:
                    dfx1 = dfx[(dfx['numeroEnvio'] == numeroEnvio)]           
                    for medicamento in dfx1['medicamento']:
                        list_medicamentos.append(medicamento)
                    lmedicamento = ','.join(str(x) for x in list_medicamentos) 
                    hour = ''.join(str(x) for x in hora) 
                    minute = ''.join(str(x) for x in minuto) 
                    if numeroEnvio == '0':
                        print('numeroEnvio == 0')
                        texto_respuesta = "Hey 👋 pasaba a recordarte que debes tomar %s a las %s:%s, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes."%(lmedicamento,hour,minute) 
                        lidAlertas = dfx1['idAlerta'].unique()
                        for idAlerta in lidAlertas:
                            print(idAlerta,'--0')
                            api.ActualizarTomaMedicamentos(str(idAlerta),'NO','NO','1')                    
                    elif numeroEnvio == '1':
                        print('numeroEnvio == 1')
                        texto_respuesta = "Me di cuenta que no has tomado %s , estaba programada para tomar a las %s:%s 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes."%(lmedicamento,hour,minute) 
                        lidAlertas = dfx1['idAlerta'].unique()
                        for idAlerta in lidAlertas:
                            api.ActualizarTomaMedicamentos(str(idAlerta),'NO','NO','2')
                    elif numeroEnvio == '2':
                        texto_respuesta = "Me di cuenta que no has tomado %s , estaba programada para tomar a las %s:%s 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes."%(lmedicamento,hour,minute) 
                        lidAlertas = dfx1['idAlerta'].unique()
                        for idAlerta in lidAlertas:
                            api.ActualizarTomaMedicamentos(str(idAlerta),'NO','NO','3')    
                    elif numeroEnvio == '3':
                        texto_respuesta = "❌ Ya pasó el tiempo previsto para la toma de %s ,si queremos lograr tu bienestar necesitamos constancia 😅"%(lmedicamento) 
                        lidAlertas = dfx1['idAlerta'].unique()
                        for idAlerta in lidAlertas:
                            api.ActualizarTomaMedicamentos(str(idAlerta),'NO','SI','3')   
                    if  int(numeroEnvio) < 3:
                        try:
                            print(telegram)
                            tl.send_botton_keyboard(str(telegram),texto_respuesta,TOKEN)
                            nametxt = 'R-' + str(telegram) 
                            print(lidAlertas)
                            lidAlerta = ','.join(str(x) for x in lidAlertas)
                            rta = "¡Genial! Estás siendo adherente %s, vamos por el camino correcto 👍"%(lmedicamento)
                            tx.registro(nametxt,lidAlerta,rta,numeroEnvio,"0","0","0","0") 
                        except:
                            pass                        
                    else:
                        nametxt = 'R-' + str(telegram) 
                        rta = "Ya pasó el tiempo, no podemos registrar la toma del medicamento.  %s"%(lmedicamento)
                        tl.send_msg_2(str(telegram),texto_respuesta,TOKEN)
#                         update.message.reply_text(rta, reply_markup=reply_markup,parse_mode = "Markdown")  
                        tx.registro(nametxt,"vencido",rta,"0","0","0","0","0") 
                        print(telegram)
#                         tl.enviar_mensaje(str(telegram), texto_respuesta,URL)  
                    print(texto_respuesta)

                    list_medicamentos = []
                    print(texto_respuesta)

    fecha = []
    fecha.append(ffinal)
    df = pd.DataFrame()
    df['fecha'] = fecha        
    df.to_csv('record_fecha.txt',index=None, header=None, sep=';')
    ahora = datetime.now()
    now = ahora.strftime("%Y-%m-%d %H:%M:%S")
    print("I'm working...",now)
    

def main():
    """Start the bot."""
    ahora = datetime.now()
    now = ahora.strftime("%Y-%m-%d %H:%M:%S")
    fecha = []
    fecha.append(now)
    df = pd.DataFrame()
    df['fecha'] = fecha
    df.to_csv('record_fecha.txt',index=None, header=None, sep=';') #, header=None     
    TOKEN,URL = crd.credentialstl()
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher
#     dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text, bot_mafe))
    dp.add_handler(CallbackQueryHandler(tl.button_end)) 
    dp.add_error_handler(error)
    updater.start_polling()
    updater.idle
    schedule.every(300).seconds.do(job)#298
    while True:
        schedule.run_pending()  
    print('Inicia Servicio Bot')    
  

    
if __name__ == '__main__':
    main()

2021-02-07 16:04:05,513 - apscheduler.scheduler - INFO - Scheduler started


Exit 1
1477550675
Exit 4
Exit:2
1285499091
Exit 4
lfechas ['2021-02-07T01:00:00.000000000' '2021-02-07T05:00:00.000000000'
 '2021-02-07T09:00:00.000000000' '2021-02-07T13:00:00.000000000'
 '2021-02-07T11:00:00.000000000' '2021-02-07T12:00:00.000000000'
 '2021-02-07T14:00:00.000000000' '2021-02-07T15:00:00.000000000'
 '2021-02-07T16:00:00.000000000' '2021-02-07T10:00:00.000000000']
numeroEnvio == 0
981 --0
956799532
[981]
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA a las 1:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA a las 1:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
numeroEnvio == 0
982 --0
956799532
[982]
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA a las 5:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA a las 5:0, es importan

2021-02-07 16:09:20,372 - urllib3.connectionpool - WARNING - Connection pool is full, discarding connection: siesservices.s4l.life


1548811411
956799532
[1096 1112 1123 1136]
Hey 👋 pasaba a recordarte que debes tomar EFAVIRENZ CÁPSULA,ABACAVIR TABLETA,AMPRENAVIR CÁPSULA,LAMIVUDINA TABLETA a las 15:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar EFAVIRENZ CÁPSULA,ABACAVIR TABLETA,AMPRENAVIR CÁPSULA,LAMIVUDINA TABLETA a las 15:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
numeroEnvio == 0
1107 --0
1131 --0
1548811411
[1107 1131]
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA,LAMIVUDINA TABLETA a las 10:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA,LAMIVUDINA TABLETA a las 10:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
numeroEnvio == 0
1109 --0
1120 --0
1133 --0
1548811411
[1109 1120 1133]
Hey 👋 pasaba a recordarte que debes tomar ABACAVIR TABLETA,AMPRENAVIR

2021-02-07 16:16:33,828 - urllib3.connectionpool - WARNING - Connection pool is full, discarding connection: siesservices.s4l.life


...

['964']
I'm working... 2021-02-07 19:40:55
I'm working... 2021-02-07 19:45:56
I'm working... 2021-02-07 19:50:57
lfechas ['2021-02-07T20:00:00.000000000']
numeroEnvio == 0
1032 --0
1045 --0
1057 --0
956799532
[1032 1045 1057]
Hey 👋 pasaba a recordarte que debes tomar ACICLOVIR TABLETA,ABACAVIR TABLETA,AMPRENAVIR CÁPSULA a las 20:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar ACICLOVIR TABLETA,ABACAVIR TABLETA,AMPRENAVIR CÁPSULA a las 20:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
lfechas ['2021-02-07T20:00:00.000000000']
numeroEnvio == 0
1069 --0
1080 --0
1093 --0
1668039623
[1069 1080 1093]
Hey 👋 pasaba a recordarte que debes tomar ACICLOVIR TABLETA,ABACAVIR TABLETA,LAMIVUDINA TABLETA a las 20:0, es importante para mejorar tu salud ☺️ Por favor confirmanos cuando te lo tomes.
Hey 👋 pasaba a recordarte que debes tomar ACICLOVIR TABLETA,ABACAVIR TABLETA,LAMIVUDINA TABLE

2021-02-07 20:26:58,316 - __main__ - WARNING - Update "{'update_id': 65174744, 'message': {'message_id': 3426, 'date': 1612747614, 'chat': {'id': 1477550675, 'type': 'private', 'username': 'Juan_Carlos_Guevara', 'first_name': 'Juan', 'last_name': 'Guevara'}, 'text': 'Toma realizada!', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 1477550675, 'first_name': 'Juan', 'is_bot': False, 'last_name': 'Guevara', 'username': 'Juan_Carlos_Guevara', 'language_code': 'es'}}}" caused error "Timed out"


['952,965']
lfechas ['2021-02-07T20:30:00.000000000']
numeroEnvio == 1
1477550675
[965]
Me di cuenta que no has tomado ABACAVIR TABLETA , estaba programada para tomar a las 20:30 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes.
Me di cuenta que no has tomado ABACAVIR TABLETA , estaba programada para tomar a las 20:30 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes.
I'm working... 2021-02-07 20:31:24
lfechas ['2021-02-07T20:30:00.000000000']
1477550675
[965]
Me di cuenta que no has tomado ABACAVIR TABLETA , estaba programada para tomar a las 20:30 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes.
Me di cuenta que no has tomado ABACAVIR TABLETA , estaba programada para tomar a las 20:30 😧 sigamos el camino hacia el éxito 🤞,Por favor confirmanos cuando te lo tomes.
I'm working... 2021-02-07 20:36:26
lfechas ['2021-02-07T20:30:00.000000000']
1477550675
❌ Ya pasó el tiempo previsto para la toma de ABACA

2021-02-07 20:41:56,941 - __main__ - WARNING - Update "{'update_id': 65174746, 'message': {'message_id': 3432, 'date': 1612748512, 'chat': {'id': 1477550675, 'type': 'private', 'username': 'Juan_Carlos_Guevara', 'first_name': 'Juan', 'last_name': 'Guevara'}, 'text': 'Toma realizada!', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 1477550675, 'first_name': 'Juan', 'is_bot': False, 'last_name': 'Guevara', 'username': 'Juan_Carlos_Guevara', 'language_code': 'es'}}}" caused error "Timed out"


['vencido']
I'm working... 2021-02-07 20:46:28
I'm working... 2021-02-07 20:51:29
